In [9]:
# Basic python import
import os
import sys
import yaml
import matplotlib.pyplot as plt
import pandas as pd

# Pytorch
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# Our custom module
sys.path.append('../')
import data_processing
import models
from evaluation import *
import submission

# Config

Ma proposition : 
Avoir un Notebook template associé à des paramètres par défaut. 
Ensuite pour chaque expérience on le duplique, et on change les valeurs des paramètres que l'on souhaite.

In [10]:
with open('../configs/default_params.yaml') as default_params_file:
      default_params = yaml.safe_load(default_params_file)
params = default_params
params

data_dir = os.path.join('..','data')

DEBUG = True

torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

device

device(type='cuda', index=0)

In [11]:
# Changer de paramètres
params["train"]["batch_size"] = 2
params["train"]

{'script_path': '../train/A_COMPLETER.py',
 'batch_size': 2,
 'epochs': 1000,
 'data_augmentation': {'samplewise_center': False,
  'samplewise_std_normalization': False,
  'rotation_range': 0,
  'width_shift_range': 0.1,
  'height_shift_range': 0.1,
  'horizontal_flip': True,
  'vertical_flip': False,
  'zoom_range': 0,
  'shear_range': 0,
  'channel_shift_range': 0,
  'featurewise_center': False,
  'zca_whitening': False}}

# Export masks as .tiff files

If it's the first time you run this notebook, you should uncomment the following cell and run it. It will read the masks from the .csv file and output them as .tiff files in a "train_masks" folder

In [12]:
from data_processing.preproc import create_masks_as_tiff, preprocess_images_and_masks

# create_masks_as_tiff(data_dir)

# Create Dataset and Dataloader

In [13]:
from data_processing.utils import get_training_datasets_and_dataloaders

train_dataset, validation_dataset, train_dataloader, validation_dataloader, encoder = get_training_datasets_and_dataloaders(batch_size=params["train"]["batch_size"] ,input_size=512)

# Train

In [14]:
from models.unet import UNet
from sklearn.preprocessing import OneHotEncoder

MODEL = UNet(num_classes=1).to(device)
lr = 1e-4
optimizer = optim.Adam(MODEL.parameters(), lr=lr)
loss = nn.MSELoss()
classif_loss = nn.CrossEntropyLoss()
p_s_classif_loss = nn.MSELoss()

optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
)

In [17]:
for X, organ, label, p_size in train_dataset:
    print(organ, p_size)
    break

tensor([0., 1., 0., 0., 0.], dtype=torch.float64) tensor(2.3438)


In [19]:
from train.train import main_train, main_train_batch1

if params["train"]["batch_size"] == 1:
    loss_list, classif_loss_list, p_s_classif_loss_list = main_train_batch1(model=MODEL.to(device), loss_fn=loss, o_classif_loss_fn=classif_loss, p_s_classif_loss_fn=p_s_classif_loss, optimizer=optimizer, n_epochs=50, dataset=train_dataset, device=device)
if params["train"]["batch_size"] > 1:
    loss_list, classif_loss_list, p_s_classif_loss_list = main_train(model=MODEL.to(device), loss_fn=loss, o_classif_loss_fn=classif_loss, p_s_classif_loss_fn=p_s_classif_loss, optimizer=optimizer, n_epochs=50, dataloader=train_dataloader, device=device)

  0%|          | 0/141 [00:00<?, ?it/s]

tensor([[-0.3086, -0.1451, -0.3237,  0.0342, -0.0397],
        [-0.1510, -0.3272,  0.0541, -0.2237,  0.0833]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([[-0.0929],
        [ 0.1501]], device='cuda:0', grad_fn=<AddmmBackward0>)


c:\Users\bapti\miniconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|▏         | 2/141 [00:25<25:43, 11.10s/it]

tensor([[ -5.3991,  -5.6756,  -5.4803,   5.5822,  -5.6317],
        [-10.4860, -10.3977, -10.7434,  10.5955, -10.4006]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([[ 5.8589],
        [11.5642]], device='cuda:0', grad_fn=<AddmmBackward0>)


  1%|▏         | 2/141 [00:26<30:30, 13.17s/it]


KeyboardInterrupt: 

### Segmentation Loss

In [ ]:
plt.plot(loss_list)

NameError: name 'plt' is not defined

### Organ Classification Loss

In [ ]:
plt.plot(classif_loss_list)

### Pixel Size Prediction Loss

In [ ]:
plt.plot(p_s_classif_loss_list)

## Save model

In [ ]:
import time
torch.save(MODEL, f"../model_save/save_{time.gmtime(time.time())}")

# Evaluation

Use evaluate_model.ipynb with saved model.

# Submission

To do with the kaggle notebook for submit.